In [0]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0430 11:03:59.920504 139636822812544 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [0]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.9MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_output'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: bert_output *****


#Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [0]:
from tensorflow import keras
import os
import re

In [0]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

error_type = "Spelling" #@param ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
error_ratio = "ToFifteen" #@param ["ToFifteen", "AugmentedRatio"]

error_type = translate_dict[error_type]

It's about time we named our model

In [0]:
modelname = "lex_item_choice_ToFifteen" # @param {type:"string"}

In [0]:
import shutil

filename = error_ratio+"/train/"+error_type+".json"
shutil.copy2('/content/gdrive/My Drive/Datasets/'+filename,'.')
trn = pd.read_json(error_type+".json").reset_index(drop=True)
trn["is_error"] = trn["is_error"].astype(int)

filename = error_ratio+"/test/"+error_type+".json"
shutil.copy2('/content/gdrive/My Drive/Datasets/'+filename,'./'+error_type+'_test.json')
tst = pd.read_json(error_type+"_test.json").reset_index(drop=True)
tst["is_error"] = tst["is_error"].astype(int)

In [0]:
len(trn)

113830

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [0]:
train = trn
test = tst

In [0]:
train.columns

Index(['context', 'is_error', 'path', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [0]:
DATA_COLUMN = 'context'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
train_InputExamples

0         <bert.run_classifier.InputExample object at 0x...
1         <bert.run_classifier.InputExample object at 0x...
2         <bert.run_classifier.InputExample object at 0x...
3         <bert.run_classifier.InputExample object at 0x...
4         <bert.run_classifier.InputExample object at 0x...
5         <bert.run_classifier.InputExample object at 0x...
6         <bert.run_classifier.InputExample object at 0x...
7         <bert.run_classifier.InputExample object at 0x...
8         <bert.run_classifier.InputExample object at 0x...
9         <bert.run_classifier.InputExample object at 0x...
10        <bert.run_classifier.InputExample object at 0x...
11        <bert.run_classifier.InputExample object at 0x...
12        <bert.run_classifier.InputExample object at 0x...
13        <bert.run_classifier.InputExample object at 0x...
14        <bert.run_classifier.InputExample object at 0x...
15        <bert.run_classifier.InputExample object at 0x...
16        <bert.run_classifier.InputExam

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'MA' and split_tokens[index + 2] == '##S' and split_tokens[index + 3] == '##K' and split_tokens[index + 4] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0430 11:04:57.207162 139636822812544 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 11:04:59.124684 139636822812544 saver.py:1483] Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [0]:
tokenizer.tokenize("This here's an example of using the BERT [MASK] tokenizer")

['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 '[MASK]',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 113830


I0430 11:05:01.549790 139636822812544 run_classifier.py:774] Writing example 0 of 113830


INFO:tensorflow:*** Example ***


I0430 11:05:01.553374 139636822812544 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0430 11:07:17.218272 139636822812544 run_classifier.py:468] label: 1 (id = 1)


INFO:tensorflow:Writing example 10000 of 28458


I0430 11:07:29.713147 139636822812544 run_classifier.py:774] Writing example 10000 of 28458


INFO:tensorflow:Writing example 20000 of 28458


I0430 11:07:41.098845 139636822812544 run_classifier.py:774] Writing example 20000 of 28458


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)

21342


In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'bert_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7eff45e3cc18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0430 11:07:50.925736 139636822812544 estimator.py:201] Using config: {'_model_dir': 'bert_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7eff45e3cc18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [27]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0430 11:08:51.356946 139636822812544 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 11:08:55.607579 139636822812544 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 21342 into bert_output/model.ckpt.


I0430 16:55:46.123734 139636822812544 basic_session_run_hooks.py:594] Saving checkpoints for 21342 into bert_output/model.ckpt.


INFO:tensorflow:Loss for final step: 5.567171e-05.


I0430 16:55:58.006764 139636822812544 estimator.py:359] Loss for final step: 5.567171e-05.


Training took time  5:48:06.960535


Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Now let's use our test data to see how well our model did:

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [29]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0430 17:00:17.115356 139636822812544 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 17:00:19.542038 139636822812544 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 21342: auc = 0.54723585, eval_accuracy = 0.9399817, f1_score = 0.170068, false_negatives = 1604.0, false_positives = 104.0, global_step = 21342, loss = 0.5148378, precision = 0.6272401, recall = 0.098369874, true_negatives = 26575.0, true_positives = 175.0


I0430 17:05:30.698723 139636822812544 estimator.py:1979] Saving dict for global step 21342: auc = 0.54723585, eval_accuracy = 0.9399817, f1_score = 0.170068, false_negatives = 1604.0, false_positives = 104.0, global_step = 21342, loss = 0.5148378, precision = 0.6272401, recall = 0.098369874, true_negatives = 26575.0, true_positives = 175.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 21342: bert_output/model.ckpt-21342


I0430 17:05:34.076090 139636822812544 estimator.py:2039] Saving 'checkpoint_path' summary for global step 21342: bert_output/model.ckpt-21342


{'auc': 0.54723585,
 'eval_accuracy': 0.9399817,
 'f1_score': 0.170068,
 'false_negatives': 1604.0,
 'false_positives': 104.0,
 'global_step': 21342,
 'loss': 0.5148378,
 'precision': 0.6272401,
 'recall': 0.098369874,
 'true_negatives': 26575.0,
 'true_positives': 175.0}

#Extracting the trained model


Now let's save our model:

In [0]:
train_InputExamples[0]

In [0]:
# Export the model
def serving_input_fn():
    label_ids = tf.placeholder(tf.int32, [None], name='label_ids')
    input_ids = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='input_ids')
    input_mask = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='input_mask')
    segment_ids = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='segment_ids')
    input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
        'label_ids': label_ids,
        'input_ids': input_ids,
        'input_mask': input_mask,
        'segment_ids': segment_ids,
    })()
    return input_fn

EXPORT_DIR = './Exported models/'+modelname
estimator._export_to_tpu = False  # this is important
path = estimator.export_savedmodel(EXPORT_DIR, serving_input_fn)
print(path)

INFO:tensorflow:Calling model_fn.


I0430 03:53:41.027125 139983797589888 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 03:53:45.399352 139983797589888 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Assets written to: ./Exported models/lex_item_choice_ToFifteen/temp-b'1556596421'/assets


I0430 03:53:46.653434 139983797589888 builder_impl.py:763] Assets written to: ./Exported models/lex_item_choice_ToFifteen/temp-b'1556596421'/assets


INFO:tensorflow:SavedModel written to: ./Exported models/lex_item_choice_ToFifteen/temp-b'1556596421'/saved_model.pb


I0430 03:53:50.890610 139983797589888 builder_impl.py:414] SavedModel written to: ./Exported models/lex_item_choice_ToFifteen/temp-b'1556596421'/saved_model.pb


b'./Exported models/lex_item_choice_ToFifteen/1556596421'


Check if we can load it correctly:

In [0]:
from pathlib import Path

subdirs = [x for x in Path(EXPORT_DIR).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

In [0]:
from tensorflow.contrib import predictor

predict_fn = predictor.from_saved_model(latest)

INFO:tensorflow:Restoring parameters from Exported models/lex_item_choice_ToFifteen/1556596421/variables/variables


I0430 03:53:56.162393 139983797589888 saver.py:1270] Restoring parameters from Exported models/lex_item_choice_ToFifteen/1556596421/variables/variables


In [0]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [38]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set[:128])
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="blue">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [45]:
import json

curname = "wsites_" + modelname + ".json"

test_texts = {"./data/exam/exam2014/EPa_38_1.txt": "The charts illustrate information on the ages of the population in both countries Yemen and Italy. It show in percentage how many people of different ages live in 2000 and how many will live in 2050. It is immediately apparent that more half people in Yemen it is children and in Italy it is people whose age between 15 and 59 years.\nYet, we can see that in Yemen percent people whose age is more 60 years, is very small. Can suppose tat Yemen is country with bad standart of living and middle age this country 60 years.\nIf look on 2050 year, at first glanne we can see that nothing change. In Yemen percent people whose age is between 15 an 59 is increased an quantity children is redused. And in Italy quantity old people is increased on 20 percents.\nFrom date it is clearly that Yemen is country with high level of borning and small continue of life. \n", "./data/exam/exam2017_4/DPe_83_1.txt": "The bar chart shows the changes of unemployment rate in some world regions in 2014 and 2015. It has six different regions starting from N. Africa to Worldwide.\nAs we can see the maximum of unemployment people is in N. Africa. The rate reached the level of 12,5 per cent. The lawest percentage of unemployment people has S. Asia region. The average data accounts for EU.\nIt can be clearly seen, that there are difference beetween the 2014 and 2015. Countries as S. Asia, N. Africa and Worldwide have stable results of unemployment trends. In contrast, the situation in Middle East, EU and Latin America is different and percentage rate is flactuating. The most sharply drop of unemployment people happend in Middle East and Latin America regions. The decline is a mere 1-2%.\nThe worst worst and the best results of unemployment rate can be explained by the high level of poverty in N. Africa and the most modern technologies and lifestyle in S. Asia.\n", "./data/Exam_practice/OV_2_year/essays_it/st_100_7.txt": "Academic achievments have become the main factor of young people's success nowadays.as the youth becomes more and more ambitious and the schedule becomes more and more busy many people are ledto the question if non-academic sbjects are needed at school.\nSome peope think that non-academic subjects such as P.E., cookery, photography and others are completely useless and should be eliminated because children need their whole learning time and concentration to improve their results, do their best and be succesful in future. And non-academic subjects are regarded as a barrier on their way to academic success.\nPersonally I completely disagree with that people, because non-academic subjects teach children qulities that must be very important in every person's life such as leadership, strengths and responsibility. Moreover young people can develop special skills that are useful in their independent everyday life (cooking, sewing etc). It is also worth saying that not every person wants to succeed academically, there is a lot of other areas which could be interesting and appreciated by young people. Some of them can't live without art, others want to succeed in sports, the third may dream about their own business.\nThat is why it is very important to give all the children the opportunity to develop their skills and talents, to do what their want to do. In addition Ican say that non-academic subjects are also important even for those who have chosen an academic career. For example, P.E. is a very good way to fight stress, to become more healthy, motivated and concentrated. \nTo put all in a nutshell, I would like to say taht even those who want to succeed academically can't go without non-academic subjects at school.\n", "./data/exam/exam2014/2012-2014_3/esl_00700.txt": "It is widely discussed whether it is necessary to include unpaid community service into school programmes. I strongly believe that it is a very efficient improvement of school programm.\nFirst, it is a simple method to educate young people. As the job they would be provided with is not paid, it could be easier to make them to get used to work or even it is possible to give rise to labour without thinking of salary.\nSecond, charity and volunteer work develops such qualities like generosity, kindness, helpfulness and capacity to manage their lives and cope with their problems as well as the others'. People become more responsible and mature. It is also probable that such experience of a work might help with the choice of one's future profession.\nThird, this may be profitable for the society itself. Many problems like with wild packs of dogs and cats, environment pollution will be resolved.\nI know many examples of how good charity and upaid work affected youth. Helping people in need, cleaning streets, helping with abandoned animals made them grow up a bit and change their view on life in general, Some can't stop helping and being volunteers as they really enjoy it. I myself had an experience of volunteering and am pretty fond of it. I find it good for socialisation, because I worked with kids from schools during olympiads and even befriended some of them.\nTo conclude, I can not find any drawbacks in introducing obligatory volunteer work into school programm.", "./data/exam/exam2017/EGe_177_2.txt": "One of the main topics of last century was space exploration. Many great discoveries were made, but are they so great that they mean more than unsolved issues on Earth? I tend to agree with the opinion that money spent on space exploration can be more useful on Earth.\nFirst of all, poverty is still a very big issue in lots of countries. There a still places where average income is about 15 dollars a month, whereas one spaceship start (not always a fortunate one) costs billions of dollars.\nThere are many other problems in the world which can be solved with money. For example, some enviromental issues, such as pollution. In China this is a very big problem. There are a lot of cities with more than a million of people, this means that there are huge numbers of cars on a small territory. This led to constant smoge on the streets which led to that almost every citizen has problems with their lungs. Although, there are solutions. For example, Singapoore builds high-tech systems to grow as many plants on their territory as they can. This is a city which is also a country, there are not much space, so they decided to use rooftops, balconies, even walls as places where trees and other oxygen makers can grow.\nOn the other hand, space exploration gives us knowledge of some fundamental laws which will help to live a better life. Some people even say that someday everyone will just leave this planet and everything will get better. As for myself, I prefer not to think what will be after myself but think about higher quality life.", "./data/exam/undefined/EEm_151_1.txt": "primary education by region and general during the period from 2000 to 2012.\nIt is clearly to see that it is similar percentage of boys and girls in Africa in 2000. Nevertheless, as you can see in the end of 2012 the number of children decrease in general.  However, it had the same percentage with South Asia in 2000. There is no a big deal of difference between the Africa in 2012 and South Asia in 2000. Going to Rest of World we should admit that general number of children felt down on ten percent.\nAccording to the chart we can see that in general the percentage of without access to primary education become lower than in 2000.  From my point of view it is good illustration of effective education program which is really work, so it is a great oportunity for children to get education in the future.\n", "./data/exam/exam2017/DOv_32_1.txt": "The graph presents the usage of three different social networks by online adults in the USA by age group.\nAs research shows the usage of Facebook strictly correlates with the age of users. People which are in the group 18-29 use facebook more than any other groups of users. Older categories of online adults in the USA have lower percentages.\nInstagram is also more popular among the youngest adults. The middle-aged group 30-49 use it twice less than young one. Similar drop happens in next groups of 50-64 and 65+. But as chart shows Instagram is not as popular as Facebook among online adults. LinkedEn shows opposite to previous results. The highest percentage of users is located in the group of 30-49. The group 50-64 decreased by 1% in comparison with middle aged group. Equal percentages are presented by 18-29 and 65%.\nOverall Facebook is the most popular social network, Instagram is popular among young group. And LinkedEn has higher demand among middle-age groups.\n", "./data/2012-2014/esl_00016.txt": "When I was a child we used to spend our holidays n the countryside. We would come to our little house on Saturday. We would cook meat and potatoes and always had our supper late in the evening. After supper we would watch TV, but one day my father bought a set of the lawn furniture, and very soon we got  used to spend our evenings in the garden. We would go to bed  midnight. My bed was considered quite uncomfortable, but I was used to  it. \nWe would go to the nearby forest  Sunday mornings. We also used to swim in a lake when the weather was fine, but, unfortunately, one day we found  that the lake had just been polluted and decided to do without swimming in it. We would spend the whole day in the open air and leave the house after sunset... Unfortunately, we had to sell our house in 2005. \n\nPlease note the \u2026-ing form after BE/GET USED TO\n\n\n\n", "./data/old IELTS/IELTS2016/MTsy_40_2.txt": "Nowadays, there is an opinion that governments should control travels by plans because of their harmful influence on the atmosphere and our enviroment. I cannot completely agree with this statement and in this essay will be examined reasons why air travels are necessary.\nFirstly, there are no doubts that plans are fastest and safest kind of transports. For instance, the distance between Moscow and Paris takes nearly 5 days by train. At the same time plan needs some hours to get the same point. This transport saves time and makes people\u2019s lives easier. Moreover, the official statistics consider that accidents among airplants happen rarer than with other transports.\nsecondly, globalisation requires to follow a high temp of life. Businessmen should make trips too often keeping their companies profitable. Besides, majority of life\u2019s spheres relate to global business therefore issues in this area can led to reducing of the life level. There are different cases than huge transport accidents were a cause of problems in the cities or even countries.\nThe last but not least reason why air travels should not be controled is existence of other sources of pollution which are more dangerous. There are amounts of manufactures which influence air and global warming more harmful than gas of airplans. I strongly believe that goverment should pay attention to pollution from earth transports and manufactures because their negative influence is well-known and does not bring usility like plans.\nIn conclusion, I consider that nowadays society cannot exist in the normal way without airplans. It is essential part of its life. Besides, goverment has an opportunity to reduce global warming and air pollution introducing laws in other spheres of society\u2019 life.\n", "./data/exam/exam2017_2/ABl_18_2.txt": "It is argued that free sharing and copy of music and films in the Internet does a lot of harm to the people who produce this content and those who enjoy it and those who do not want to pay must be considered as criminals. Personally I disagree with that claim and I hold the opinion that punishment is not an effective solution to the named problem.\nUndoubtely, there are some reasons for punishing people who do not pay for the content. To begin with, musicians and producers lose money they deserved and it leads to lack of motivation to create new films and music tracks. Additionally, money payed for art producers are investments in future projects. Finally, losing such amounts of money hurts not only creators, but also the government budget which has its own profit from the taxes.\nNevertheless, punishing for copying and sharing music and videos seems to be bad idea. Firstly, free products of art provides cultural development of the population. It is irrational and even unfair to punish those who are aimed at self-development but do not have money for it. This person can be a child, a student or just somebody with financial problems. Secondly, people who are able to pay will pay if they adore the content and they want to say some thanks to those who created it. Thirdly, the Internet is the informational field which is difficult to control by the law, so punishing will be technically difficult to implement.\nTo sum up, punishing for sharing and copying objects of art can satisfy those who produce them, but also can influence on cultural development in negative way.\n", "./data/old IELTS/IELTS2016/EEm_200_1.txt": "In the given chart we can see the quantity of children who don\u2019t have access to attend classes in primary school. The information is devided by gender and regions from 2000 to 2012.\nThe chart below shows that the higher level of children without education was in Africa in 2000. The number of such boys and girls was nearly the same and and was 43.7 millions. 12 years have passed and the number reduced and became 32.7. Africa showed the same level in 2012 as South Asia in 2000. But looking at the whole chart it is possible to notice that the lowest number of childer without access to primary school by the year 2012 is in South Asia, althouh in the privious years it\u2019s level  (32.2) in general was higher than in the rest of the world (23.3).\nIn conclusion I would like to say that comparing with such countries as South Asia and Africa, the rest of the world showed better results. And I would like to add that in general girls have less opportunities for education than boys.\n", "./data/exam/exam2017_6/OBy_5_2.txt": "A lot of people consider, that regulary using if electronic devices can cause problems with health. Usually, it can be caused by the way we use them and the amount of time, which we spend on them.\n\nFirst of all, light of the computer and  telephone screen are hurt people\u2019s eye sight. A lot of people use mobile phones and smartphone everyday and everywhere, including the time before sleeping. Preparing for seeping is very important thing, because all this time our body (including eyes) relaxs. However, using the phone before going to bed makes our eys work, which can cause the decreasing of the eye-sight. In addition, it can cause problems with sleeping, due to the fact, that our brain can\u2019t relax too. This problem can be solved by making some exercizes for eyes every hour after using smartphone or computer. Also, people should reduce time, which is spent for devices.\n\nThe second problem can be caused by the wrong position in front of a computer. Nowadays, people tend to spent every day at work sitting on the computer chair. Almoust everybody can\u2019t choose the right position to seat on it, which causes problems with the spine. Hopefully, this problem can be solved by physical exercizes. People should do them after every 45 minutes being in front of the computer screen or in the gym.\n\nAll in all, despite the benefits of modern technology, people should think about their health. These problems can be reduced by replacing the light screens to alternative type of getting information as book or newspaper. \n", "./data/old IELTS/IELTS2015/ESha_49_2.txt": "As it is commonly understood to day the level of public health plays a significant role in the developing of society. One of the most appropriate way of improving public health is to provide people with more sport facilities. However, some people believe that such kind of method is not effective and that other solutions to the problem should be found. Personally, I am sure that increasing the number of sports facilities, in fact, can improve public health.\n\n\nFirst of all, there is a clear connection between the quantity of sports facilities and public health. It is quite obvious that if there are more sports facilities there will be more opportunities to improve health. What is more, we should take into consideration the fact that people are influenced by the advertisement and that is why it is not so complicated to make people want to use sports facilities more often. Last but not the least is the point that  the big number of sports facilities will undoubtly lead to the improvement of the level of service, equipment and staff. Moreover, such situation is positive because of the fact that sports facilities will be more reliable and convinient.\n\n\nBut others reckon that he increasing the number of sports facilities will not lead to the appropriate result. First of all, healthy way of living is not so significant in society nowadays. Furthermore, it is quite complicated to provide society with sports facilities. Finally, there are other factors that influence public health and of course methods of improving are different.\n\n\nIn conclusion, I would like to reckon that while the problems concerning public health are existing different methods of its solving will appear. As any other method the increasing the number of sport facilities has its advantages and drawbacks. But as for me, I guess that such kind of method is quite effective.\n", "./data/2012-2014/esl_00749.txt": "That encounter happened to me a week ago when I was taking a tram home from the shopping mall. I was taking to my Mum when saw a familiar face in the crowd but I couldn\u2019t understand where I could have seen this face earlier. I was nearly shocked when this person greeted me and completely horrified when I understood that this person was in the uniform. I hope that this messy train of thoughts wasn\u2019t shown on my face. Only after this person came closer I understood that it was my former classmate whom I haven\u2019t seen for about 3 years and a half. He was wearing uniform because that day he had a military chair at his university. My classmate and I got off the tram leaving my Mum alone as she had to collect my brother from school. We live nearly close to each other and we walked our ways home talking about all the events happened with us when we were studying together. The weather was wonderful and it was such a nice experience to discuss our memories from the past.\n", "./data/exam/exam2016/JSl_87_1.txt": "The graph presents the information about changes in investment in renewable energy during the period from 2006 to 2013 in two kinds of countries: Developed and Developing. As it could be seen from the graph, the investments in Developed Countries was slightly increasing during three years period. The investments was reaching about 100 billion of dollars, meanwhile investments in developing countries was also steadily increasing. In the ear 2011 investments in Developed Countries reached the peak with more then 150 billions of dollars. However, the peak of Developing Countries was in the year 2012. In the year 2011 the amount of investments in developed countries started to declining, while investments in Developed Countries was still growing steadily. Let us have a look at  the graph od World Total investments. Obviously, the year 2011 was the peak of all investments in the world. To sum up, we can name the main trend of the graph which is increasing of investments. However, from the year 2011 there were a slight decreasing of investments in Developed Countries. The year 2013 also should be mentioned, because there where a decline of investments in Developing Countries. \n"}

outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 128


I0430 17:12:30.266695 139636822812544 run_classifier.py:774] Writing example 0 of 128


INFO:tensorflow:*** Example ***


I0430 17:12:30.271303 139636822812544 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0430 17:14:41.667208 139636822812544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0430 17:14:41.772185 139636822812544 session_manager.py:493] Done running local_init_op.




Annotation took time  0:00:08.226041


In [0]:
entities = annotate(test_texts["DTi_50_2"])

INFO:tensorflow:Writing example 0 of 290


I0430 03:16:32.357002 139983797589888 run_classifier.py:774] Writing example 0 of 290


INFO:tensorflow:*** Example ***


I0430 03:16:32.360667 139983797589888 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0430 03:16:32.452672 139983797589888 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0430 03:16:32.454799 139983797589888 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0430 03:16:32.457265 139983797589888 run_classifier.py:468] label: 0 (id = 0)


In [0]:
entities[0].segment_ids

In [0]:
import numpy as np

predict_fn({
        'label_ids': [0, 1],
        'input_ids': entities[0].input_ids,
        'input_mask': entities[0].input_mask,
        'segment_ids': entities[0].segment_ids
    })

ValueError: ignored

In [0]:
s_ids = np.array([entities[0].segment_ids for _ in range(BATCH_SIZE)])
s_ids.shape

(32, 128)

In [47]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [48]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

W0430 17:17:09.310078 139636822812544 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNo

Uploaded file with ID 1g_KEmaT8KldhnNvTBoR0z4RehZt7LIYX


In [0]:
# Create & upload a file.

os.chdir("bert_output")
for name in ['model.ckpt-21342.data-00000-of-00001', 'model.ckpt-21342.meta', 'model.ckpt-21342.index']:
  uploaded = drive.CreateFile({'title': name})
  uploaded.SetContentFile(name)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1D3yH2qc2kaVMIYvFfZRax86ZqHGeNfR-


In [50]:
os.listdir("./bert_output")

['eval',
 'model.ckpt-21342.meta',
 'model.ckpt-21000.index',
 'model.ckpt-20500.index',
 'model.ckpt-21342.data-00000-of-00001',
 'model.ckpt-20000.meta',
 'model.ckpt-21000.meta',
 'model.ckpt-20000.data-00000-of-00001',
 'model.ckpt-19500.index',
 'model.ckpt-21000.data-00000-of-00001',
 'graph.pbtxt',
 'model.ckpt-19500.data-00000-of-00001',
 'checkpoint',
 'model.ckpt-20000.index',
 'model.ckpt-20500.data-00000-of-00001',
 'model.ckpt-19500.meta',
 'model.ckpt-21342.index',
 'events.out.tfevents.1556622549.987caecf3296',
 'model.ckpt-20500.meta']

# Preparing existing model for REALEC production